### I. INTRODUCTION 

__Context__: The Albarka program, implemented by Save the Children in Mali and funded by the US Bureau of Humanitarian Assistance (BHA), aims to encourage good nutrition, health, and hygiene practices. One of the ways these behaviors are promoted is through the GSAN (Groupes de Soutien aux Activités de Nutrition) platform.

__Questions__: 

* What is the proportion of women participating in GSAN?
  
* How much has participating in GSAN impacted women's diet diversity?

* How does region, as a variable,  influence the likelihood of women meeting the minimum dietary diversity ?
  
* To what extent does the level of education influence women's diet diversity?

__Data Source__: We will use data from the Albarka 2023 annual survey to create a dataset with four key variables:

* mwdd: A 0/1 (0= no, 1= yes) variable showing whether a woman consumed at least five food groups in the last 24 hours.

* gsan_factor: A yes/no variable showing whether a woman participated in a GSAN in the past 12 months.

* Region: The region in Mali where the woman lives.

* Cercle: Cercle in Mali where the woman lives 

* A15:  Categorical variable for Education 

__Method__:

* _Statistical Analysis_: We'll  generate descriptive statistic via the EDA steps and then we will test if there's a significant difference in the proportion of women with minimum diet diversity between those who participated in GSAN and those who did not.

* _Supervised Machine Learning_: We will treat this as a classification problem, where the goal is to predict a category—specifically, whether a woman consumes a minimum diet (1 = yes, 0 = no) based on her participation in GSAN, region and education. To do this, we'll use logistic regression, a method commonly used for classification tasks. Logistic regression helps us predict whether something belongs to one category or another based on input data. In this case, we'll use it to predict whether a woman meets the minimum diet diversity based on her participation in GSAN, region and education.

__Steps__: We will follow a six-step process. First, we'll load the necessary library and dataset to get everything ready for analysis. Next, we'll explore the data to understand its characteristics and uncover any patterns or trends, a step known as Exploratory Data Analysis (EDA). After that, we'll run a statistical test to determine if the proportions are significantly different, helping us understand any key differences in our data.

In the fourth step, we'll prepare the data by encoding categorical variables, splitting the data into training and testing sets, and setting the _features_ (GSAN participation, region and education) that we'll focus on in our analysis. Once the data is prepared, we'll run a logistic regression algorithm to analyze the relationship between the features and our outcome of interest.Finally, we'll test the models to evaluate their performance and make any necessary adjustments to improve their accuracy and reliability.

### II. Libraries and dataset loading 

A library in programming is a collection of pre-written code that provides specific functionality, making it easier to perform common tasks without having to write the code from scratch. In Python, libraries are often packages that I can import into my project to use their features.

Here are the libraries that we will be loading  and using in this project: 

__numpy__ : For numerical computing. It provides support for arrays, matrices, and many mathematical functions to operate on these data structures efficiently.

__pandas__ : For data manipulation and analysis. It provides data structures like DataFrames, which are similar to tables in a database or Excel spreadsheet, making it easier to work with structured data.


__statsmodel__ : This is a library that provides classes and functions for the estimation of many different statistical models. It also includes tools for statistical tests and data exploration.


__sklearn__: This is a popular machine learning package in Python.  It offers a comprehensive libraries that covers many aspects of machine learning, from data preprocessing to model selection and evaluation, making it a go-to tool for both beginners and experts in the field. For that project, we will import 1)  _LabelEncoder_ and OneHotEncoder for converting categorical labels into numeric values that can be used in machine learning algorithms 2) _train_test_split_ fuction to split our dataset into two parts: training data and testing data 3) _LogisticRegression_ class to implement the logistic regression and 4) _metrics.classification_report_ function to evaluate the model performance after testing it by comparing the predicted labels to the actual labels in the test data.

In [3]:
# Importing the necessary Libraries 
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report

In [4]:
# Retreiving the data and printing the first five data set rows
path = "C:/Users/Nael/Desktop/Albarka/AS2023/data/mwdd_gsan.csv"
df = pd.read_csv(path)


### III. Exploratory Data Analysis (EDA) 

We are conducting an Exploratory Data Analysis (EDA) because it is important in machine learning as it helps us to understand our data before we start building logistic model. It’is crucial for five reasons : _1) we  get to know my data_: By exploring our dataset,  we can  see what it looks like, including the number of rows and columns, types of data (like numbers or text), and any patterns or trends.  _2) we identify issues_: we can spot problems such as missing values, outliers (unusual values), or errors. If these issues are not addressed, they can cause my model to perform poorly.  _3) we can  find Relationships_:  It  helps us discover relationships between different variables.  _4) we Make Informed Decisions_: we will make better decisions about how to prepare the data for the logistic Model. This will include how to transform variables. _5)  we avoid surprises_: If we skip EDA, we might run into unexpected issues later in the modeling process. It will help us to catch these potential problems early, saving time and effort.

Through this EDA, we will ask seven questions.

__1. Is the dataset in tabular format (row and column)?__ 

In [7]:
df.head()     # viewing the first 5 rows of my dataset

,mwdd,gsan_factor,Region,A15,Cercle
0,0,no,Gao,Aucun,Gao
1,0,no,Gao,Ecole coranique,Ansongo
2,0,no,Gao,Aucun,Gao
3,0,no,Gao,Aucun,Gao
4,0,no,Gao,Aucun,Gao


__2. What is the size of the dataset ?__ 

In [9]:
 print("Number of rows is:",df.shape[0]," and number of columns is: ", df.shape[1] )

Number of rows is: 683  and number of columns is:  5


__3. What is the type of variable in each column and are there missing values ?__

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 683 entries, 0 to 682
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   mwdd         683 non-null    int64 
 1   gsan_factor  683 non-null    object
 2   Region       683 non-null    object
 3   A15          683 non-null    object
 4   Cercle       683 non-null    object
dtypes: int64(1), object(4)
memory usage: 26.8+ KB


__4.What are the unique values in each column ?__

In [13]:
# Get unique values for each column
for column in df.columns:
    unique_values = df[column].unique()
    print(f"Unique values in {column}: {unique_values}")

Unique values in mwdd: [0 1]
Unique values in gsan_factor: ['no' 'yes']
Unique values in Region: ['Gao' 'Tombouctou' 'Bandiagara' 'Douentza']
Unique values in A15: ['Aucun' 'Ecole coranique' 'Primaire' 'Alphabétisé' 'Secondaire'
 'Universitaire']
Unique values in Cercle: ['Gao' 'Ansongo' 'Tombouctou' 'Koro' 'Douentza']


__5. What is the proportin of women participating in GSAN (gsan_factor class distribution) ?__

In [15]:
class_counts = df['gsan_factor'].value_counts(normalize ="index")
class_counts

gsan_factor
no     0.648609
yes    0.351391
Name: proportion, dtype: float64

__6. How many women with Minimun diet diversity are participating (or not) in GSAN ?__

In [17]:
contingency_table = pd.crosstab(df['mwdd'],             # Create a row contingency table
                                df['gsan_factor'],
                                normalize  ='index') 
print(contingency_table)                                # Print the table 

gsan_factor        no       yes
mwdd                           
0            0.764000  0.236000
1            0.581986  0.418014


In [18]:
contingency_table = pd.crosstab(df['gsan_factor'],      # Create a column contingency table
                                df['mwdd'],
                               normalize  ='index') 
print(contingency_table) 

mwdd                0         1
gsan_factor                    
no           0.431151  0.568849
yes          0.245833  0.754167


__7. How many women have met (or did not) the Minimum diet diversity (mwdd class distribution) ?__ 

In [20]:
class_counts = df['mwdd'].value_counts(normalize ="index")
class_counts

mwdd
1    0.633968
0    0.366032
Name: proportion, dtype: float64

__8. Is there a class imbalance in mwdd column ?__

In [22]:
imbalance_ratio = class_counts.max() / class_counts.min()
print(f"Imbalance Ratio: {imbalance_ratio:.2f}")

Imbalance Ratio: 1.73


__CORRECTIVE ACTIONS NEEDED IN THE LOOP OF THE EDA__ 

* The dataset consists of 2,049 data points, organized in rows and columns, with no need for missing value treatment.

* All entries are consistent, with no values falling outside the expected range.

* The columns for GSAN and Region is  currently recorded as text, and it will need to be converted to numerical binary (1,0) values before applying any machine learning algorithms.

* Region is also as a text with different value  into a signle column, we will also to need to convert so that  where each region will be  converted into its own binary (0,1) column. 

* The mwdd variable is already numeric, so no recoding is necessary. 

 * mwdd variable  displays an imbalance ratio of 1.73, meaning that one class is 1.73 times more prevalent than the other. While this ratio indicates some degree of class imbalance, it is moderate compared to more extreme cases, such as a 9:1 or 10:1 ratio. In logistic regression, such a moderate imbalance might introduce a slight bias toward the majority class, but it is unlikely to severely affect the model's performance. Nevertheless, during model evaluation, it will be  crucial to monitor metrics like precision, recall, and F1-score for both classes. This will help ensure that the model performs well across all classes and does not underperform on the minority class, which could otherwise lead to a less accurate or biased outcome.

 * A15 is an object data type.  It needs to be converted to a categorical ordinal variable  such as no education is recordied as  1, basic literacy as 2, coranic school as 3,  primary school as 4, Secondary school as 5 and university as 6.

 

### IV. Testing the difference in the proportion of women that meet the minimum diet diversity 

A test of significant difference in proportions is a statistical test used to determine whether the difference between the proportions of two groups is statistically significant. This kind of test is particularly useful when comparing categorical data, where the outcome is typically binary (e.g., success/failure, yes/no). We will perform the Z-Test for Proportions which is the most common test used with the following stated  Hypotheses:

* 	Null Hypothesis (H₀): There is no difference in the proportions of women who consumed a minimum diet diversity between those participating in GSAN and those who did no (p1 equal p2)
  
*  Alternative Hypothesis (H₁): There is a difference between the proportions (p1≠p2).
  
We will a reject the  null hypothesis if P-Value is less than the significance level (typically 0.05),  indicating a significant difference in proportions .


In [25]:
# Create a column contingency table 
contingency_table = pd.crosstab(df['gsan_factor'], df['mwdd'])

# Extract the counts from the table
count = contingency_table.loc['yes', 1]  # Number of '1' in mwdd for 'yes' in gsan_factor
nobs = contingency_table.loc['yes'].sum()  # Total number of observations for 'yes' in gsan_factor

count_2 = contingency_table.loc['no', 1]  # Number of '1' in mwdd for 'no' in gsan_factor
nobs_2 = contingency_table.loc['no'].sum()  # Total number of observations for 'no' in gsan_factor

# Perform the Z-Test
stat, p_value = sm.stats.proportions_ztest([count, count_2], [nobs, nobs_2])

# Output the results
print(f"Z-Statistic: {stat}")
print(f"P-Value: {p_value}")

Z-Statistic: 4.799779347351392
P-Value: 1.5884053722939712e-06


__FINDINGS FROM THE TEST:__ 
The results we've obtained from the Z-test indicate the following:

* Z-Statistic: 4.799779347351392: This value suggests that the observed difference in proportions is about 4.8 standard deviations away from the expected difference under the null hypothesis. A higher absolute Z-value indicates a more significant deviation from the null hypothesis.

* P-Value: 1.5884053722939712e-06: This p-value is extremely small (approximately 0.000001588), much less than the common significance level of 0.05. This means that the null hypothesis (which states that there is no difference in proportions between the two groups) can be rejected with strong confidence.

_Interpretation:_
There is a statistically significant difference in the proportions of women with minimum diet  between the groups defined gsan_factor (participating or not pariticpating in GSAN. This means that participation in GSAN (as indicated by gsan_factor) has a significant association with whether women meet the minimum diet diversity requirement (mwdd).

### V. Data preparation and  Features ingineering 

Feature engineering is the process of selecting, modifying, or creating new features (or variables) from raw data to improve the performance of machine learning models. Features are the input variables that the model uses to make predictions, and the quality and relevance of these features can significantly impact the model's accuracy and effectiveness. In our project and based on the findings of the EDA, we will limit the features engineering to : 

* _Feature Selection_:  selection the predictor (gsan_factor) and the depend variable (mwdd).
   
* _Feature Transformation_: Given the nature of the variable, there is no need to scaling, normalization, log transformation, but as the EDA showed we will need to do encoding (e.g., converting categories into numerical values) for gsan and region.  We will specifically do one-hot-encoding which consists of converts each category into a new binary column (0 or 1) for each unique category value.

As we are building a logistic regression model (or  with any machine learning model), we want to ensure that the model generalizes well to new, unseen data. Therefore spliting the dataset in to training and testing set will be a crucial in preparing our data. The model will learn patterns from the training set. It will adjust its parameters to minimize errors on this data. After the model is trained, we will evaluate it on the testing set, which contains data the model hasn't seen before. This step helps us to asses how well the model is likely to perform on new, real-world data.
Without a separate testing set, we might overestimate the model's performance because it would have "seen" all the data during training.


__Creating a new column for livelihood zone based on the cercle__

In [29]:
# Nested if-then using np.where
df['LZ_cercle'] = np.where(df['Cercle'] == "Gao", "ML02_ML03_ML04",
                    np.where(df['Cercle'] == "Ansongo", "ML02_ML03_ML04",
                    np.where(df['Cercle'] == "Tombouctou", "ML02_ML03",
                    np.where(df['Cercle'] == "Koro", "ML13_ML09",
                    np.where(df['Cercle'] == "Douentza", "ML13", "Unknown")))))

__Encoding gsan_factor and region column__ 


In [30]:
# Initialize OneHotEncoder
encoder = OneHotEncoder(drop='first', sparse=False)

# Fit and transform the 'gsan_factor' and 'Region' columns
encoded_features = encoder.fit_transform(df[['Region', 'gsan_factor','LZ_cercle']])

# Create DataFrame for encoded variables
encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(['Region', 'gsan_factor','LZ_cercle']))

# Concatenate the encoded variables back to the main DataFrame
df_encoded = pd.concat([df.drop(['Region', 'gsan_factor','LZ_cercle'], axis=1), encoded_df], axis=1)

# Display first few rows of the encoded DataFrame


C:\Users\Nael\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


__Encoding A15 (education) as a categorical ordinal__

In [32]:
# Define the mapping for ordinal categories
category_mapping = {
    'Aucun': 1,
    'Alphabétisé': 2,    
    'Ecole coranique': 3,    
    'Primaire': 4,
    'Secondaire': 5,
    'Universitaire': 6
}

# Convert the column to categorical with the specified order
df_encoded['A15'] = df['A15'].map(category_mapping).astype('category')

In [33]:
df_encoded.head()

,mwdd,A15,Cercle,Region_Douentza,Region_Gao,Region_Tombouctou,gsan_factor_yes,LZ_cercle_ML02_ML03_ML04,LZ_cercle_ML13,LZ_cercle_ML13_ML09
0,0,1,Gao,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,0,3,Ansongo,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,0,1,Gao,0.0,1.0,0.0,0.0,1.0,0.0,0.0
3,0,1,Gao,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,0,1,Gao,0.0,1.0,0.0,0.0,1.0,0.0,0.0


__Feature selection and split the dataset into 70% for training__

In [35]:
X = np.asarray(df_encoded[["gsan_factor_yes"]])  # gsan_factor as predictor 
y = np.asarray(df_encoded['mwdd'])           # mwdd as dependent 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [36]:
# Checking the size of the training set 
print(" The training dataset has: " ,X_train.shape [0], "rows which represent 70% of the rows from the entire dataset") 

 The training dataset has:  478 rows which represent 70% of the rows from the entire dataset


In [37]:
# Checking the  occurrences of each class of mwdd in the training set 
unique, counts = np.unique(y_train, return_counts=True)
class_distribution = dict(zip(unique, counts))

print(class_distribution)

{0: 165, 1: 313}


### VI. Fitting the Logistic regression to our training set  with only GSAN as a predictor of mwdd

Now that we have prepared the data and set the feature, we will fit the model for defining the causality and relationship for mwdd.  We therefore assume the following logistic model

logit(P(mwdd=1))= β0 + β1 × gsan_factor_yes

We will be fitting the logistic regression using two libraries: Statmodels  and scikit-learn. The first one will generate the model with coefficients for each predictor and the p-value while the second will generate the coefficients and is mostly use for prediction and model evaluation. There will be slight differences in the estimation of the predictors between statsmodels and scikit-learn when performing logistic regression. These differences are due in how these libraries handle certain aspects  such as optimization algorithms, convergence criteria  and regularization. 

__Fitting  the logistic model using  Statsmodels__

In [40]:
X_train_with_const = sm.add_constant(X_train)  
logit_model = sm.Logit(y_train, X_train_with_const)
result = logit_model.fit()

Optimization terminated successfully.
         Current function value: 0.627911
         Iterations 5


In [41]:
print(result.summary())

                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  478
Model:                          Logit   Df Residuals:                      476
Method:                           MLE   Df Model:                            1
Date:                Wed, 04 Sep 2024   Pseudo R-squ.:                 0.02561
Time:                        10:09:29   Log-Likelihood:                -300.14
converged:                       True   LL-Null:                       -308.03
Covariance Type:            nonrobust   LLR p-value:                 7.114e-05
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3762      0.115      3.264      0.001       0.150       0.602
x1             0.8383      0.218      3.850      0.000       0.412       1.265


__Fitting the logistic model using scikit-learn__

In [43]:
# Fit the logistic regression model with training set
lr = LogisticRegression(solver='liblinear').fit(X_train, y_train)
lr

LogisticRegression(solver='liblinear')

In [44]:
# Get the coefficients and intercept
coefficients = lr.coef_
intercept = lr.intercept_

# Print coefficients and intercept
print("Coefficients:", coefficients)
print("Intercept:", intercept)

Coefficients: [[0.80536063]]
Intercept: [0.38179638]


__What the fitted logistic regression reveals?__ 

The results from the Statsmodels analysis indicate that the predictor x1 (participation in GSAN) has a positive and significant effect (Causality) on the likelihood of the outcome y being 1 (meeting the minimum diet diversity). The very small p-value (P>|z| = 0.000) confirms that the effect of GSAN participation on meeting the minimum diet diversity is statistically significant, providing strong evidence that  participaiton in  GSAN (x1) is a meaningful predictor of meeting the minimun diet (y).

The coefficient for participating in GSAN  (0.8383) represents the change in the log-odds in the minimun diet diversity (y = 1) for each one-unit increase, with all other variables held constant. When we exponentiate this coefficient, we obtain an odds ratio of 2.31, suggesting that being a member of GSAN makes a woman approximately 2.23 times more likely to meet the minimum diet diversity compared to non-members.

In comparison, the Scikit-Learn analysis yielded a coefficient for participating in GSAN of 0.8053, the probability of meeting the minimum diet diversity  increases by approximately 2.71 times.

### VII. Evaluation of the  Logistic regression with only GSAN as a predictor of mwdd

To evaluate the model, we will first  make Predictions: The logitic model (lr)  will run on the test data (X_test) to predict the if the minimun diet diversity is met (y_pred).  We will then generate a classification report for comparing the predicted minimun diet diversity met (y_pred) with the actual minimun diet diversity met (y_test). The classification report will includes metrics like precision, recall, f1-score and accuracy.   Precision is the proportion of true positive predictions (correctly predicted positive instances) out of all positive predictions made by the model.Recall is the proportion of true positive predictions out of all actual positive instances. The F1-score is the harmonic mean of precision and recall. It provides a single metric that balances both precision and recall. Accuracy is the proportion of correct predictions (both true positives and true negatives) out of all predictions made by the model.

In [48]:
# Predict uisng  the test set
y_pred = lr.predict(X_test)
# Generate the classification report
report = classification_report(y_test, y_pred)
# Print the report
print(report)

C:\Users\Nael\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Nael\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Nael\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00        85
           1       0.59      1.00      0.74       120

    accuracy                           0.59       205
   macro avg       0.29      0.50      0.37       205
weighted avg       0.34      0.59      0.43       205



__What evaluation report reveals?__

The logistic regression model is struggling to correctly predict the negative class (class 0/ not meeting minimum diet diversity). Precision for class 0 is 0.00. This means that none of the instances predicted as class 0 were actually class 0.
Recall for class 0is  0.00, this means that the model didn't correctly identify any of the true instances of class 0.
F1-Score for class 0 is 0.00.  Since both precision and recall are 0, the F1-score is also 0, indicating poor performance on class 0.

The report shows a significant imbalance for each class (85 for class 0/not meeting minimun diet and 120 for class 1/meeting minimun diet). This imbalance could be causing the model to favor predicting the positive class (1) over the negative class (0), leading to poor performance on class 0. This is something we said, from the EDA step, that we would pay attention to. 

To address this Class Imbalance , we will use the  Resampling Techniques. When dealing with imbalanced classes in a dataset, resampling techniques are used to adjust the distribution of the classes, making the dataset more balanced. This helps to improve the performance of machine learning models, which might otherwise be biased towards the majority class. Specifically, we will  use oversampling (like SMOTE) for the minority class or undersampling for the majority class to balance the dataset. Using oversampling techniques like SMOTE (Synthetic Minority Over-sampling Technique) is a great way to address class imbalance. SMOTE works by generating synthetic samples for the minority class, which helps the model learn from a more balanced dataset.

__Applying SMOTE for balanced minimun diet diversity class__

In [51]:
# loading the SMOTE function from imblearn library
from imblearn.over_sampling import SMOTE

In [52]:
# Applying  SMOTE to the training data
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

In [53]:
# Checking the occurrences of each class of mwdd in the training set after after applying resampling  
unique, counts = np.unique(y_train_res, return_counts=True)
class_distribution = dict(zip(unique, counts))

print(class_distribution)

{0: 313, 1: 313}


__Fitting the logistic model using Statsmodels on the resampled training set__

In [55]:
X_train_with_const_res = sm.add_constant(X_train_res)  
logit_model_res = sm.Logit(y_train_res, X_train_with_const_res)
result_res = logit_model_res.fit()

Optimization terminated successfully.
         Current function value: 0.675968
         Iterations 4


In [56]:
print(result_res.summary())

                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  626
Model:                          Logit   Df Residuals:                      624
Method:                           MLE   Df Model:                            1
Date:                Wed, 04 Sep 2024   Pseudo R-squ.:                 0.02478
Time:                        10:09:30   Log-Likelihood:                -423.16
converged:                       True   LL-Null:                       -433.91
Covariance Type:            nonrobust   LLR p-value:                 3.522e-06
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.2561      0.098     -2.615      0.009      -0.448      -0.064
x1             0.8041      0.176      4.573      0.000       0.459       1.149


__Fitting the logistic model using scikit-learn on the resampled training set__

In [58]:
lr1_res = LogisticRegression(solver='liblinear')
lr1_res.fit(X_train_res, y_train_res)

LogisticRegression(solver='liblinear')

In [59]:
# Get the coefficients and intercept for resampled data 
coefficients = lr1_res.coef_
intercept = lr1_res.intercept_
# Print coefficients and intercept for the resampled data
print("Coefficients:", coefficients)
print("Intercept:", intercept)

Coefficients: [[0.77769246]]
Intercept: [-0.24629418]


__Evaluating logistic model fitted to the resampled training set__

In [61]:
# Predict on the test set
y_pred_res = lr1_res.predict(X_test)
# Evaluate the model
print(classification_report(y_test, y_pred_res))

              precision    recall  f1-score   support

           0       0.49      0.75      0.59        85
           1       0.72      0.44      0.55       120

    accuracy                           0.57       205
   macro avg       0.60      0.60      0.57       205
weighted avg       0.62      0.57      0.57       205



__What the fitted model and evaluation report show when using the resampled dataset ?__

 After resampling the data, the model's performance has improved, particularly in predicting class 0. The results continue to indicate that women's participation in GSAN has a positive and significant effect on the likelihood of meeting the minimum diet diversity. The odds ratio can be calculated as exp(0.0.7776) ≈ 2.71, suggesting that being a member of GSAN makes a woman approximately 2.71 times more likely to meet the minimum diet diversity compared to non-members. With the resampled data, the intercept went from a positive value to a negative (-0.2462) with odd of 0.7817. This means that the likelihood of meeting the minimum diet diversity is about 78,17% of the likelihood of not meeting the minimun diet diversity when the women is not participating in GSAN (all predictors are set to zero). 

The model stands as:  logit(P(mwdd=1))= -0.24629418 +  0.77769246 gsan_factor_yes

Though we have sufficient evidence for responding the questions in the project, we noticed an accuracy of 0.57, meaning the model is correctly classifying only 57% of the instances. This is not particularly high, especially if the classes are balanced, which suggests the model has room for improvement. We should consider now adding new feature in the model.  Let us add region to see how it changes the model.  

### IX. Logistic regression with  GSAN  and region as a predictor of mwdd

In [65]:
X1 = np.asarray(df_encoded[["gsan_factor_yes","Region_Douentza","Region_Gao","Region_Tombouctou"]])  # gsan_factor as predictor 
y1 = np.asarray(df_encoded['mwdd'])           # mwdd as dependent 

In [66]:
#  Alternative code for Defining  feature matrix X and target vector y
# X1 = df_encoded.drop('mwdd', axis=1)
# y1 = df_encoded['mwdd']

In [67]:
# Split the dataset
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.3, random_state=42)

In [68]:
X_train_res1, y_train_res1 = smote.fit_resample(X_train1, y_train1)

In [69]:
X_train_with_const_res1 = sm.add_constant(X_train_res1)  
logit_model_res1 = sm.Logit(y_train_res1, X_train_with_const_res1)
result_res1 = logit_model_res1.fit()

Optimization terminated successfully.
         Current function value: 0.520393
         Iterations 7


In [70]:
print(result_res1.summary())

                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  626
Model:                          Logit   Df Residuals:                      621
Method:                           MLE   Df Model:                            4
Date:                Wed, 04 Sep 2024   Pseudo R-squ.:                  0.2492
Time:                        10:09:31   Log-Likelihood:                -325.77
converged:                       True   LL-Null:                       -433.91
Covariance Type:            nonrobust   LLR p-value:                 1.179e-45
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.5886      0.170     -3.470      0.001      -0.921      -0.256
x1             0.1314      0.237      0.554      0.579      -0.333       0.596
x2             2.6981      0.287      9.416      0.0

In [71]:
lr1_res1 = LogisticRegression(solver='liblinear')
lr1_res1.fit(X_train_res1, y_train_res1)

LogisticRegression(solver='liblinear')

In [72]:
# Get the coefficients and intercept for resampled data 
coefficients = lr1_res1.coef_
intercept = lr1_res1.intercept_
# Print coefficients and intercept for the resampled data
print("Coefficients:", coefficients)
print("Intercept:", intercept)

Coefficients: [[ 0.15320288  2.52148752  0.25779491 -1.95838839]]
Intercept: [-0.57054151]


In [73]:
# Predict on the test set
y_pred_res1 = lr1_res1.predict(X_test1)
# Evaluate the model
print(classification_report(y_test1, y_pred_res1))

              precision    recall  f1-score   support

           0       0.58      0.98      0.72        85
           1       0.97      0.49      0.65       120

    accuracy                           0.69       205
   macro avg       0.77      0.73      0.69       205
weighted avg       0.81      0.69      0.68       205



__What the results show when gsan and  region are used together as a predictor in meeting minimun diet ?__

_Effect of Adding Region Variables_: The significant drop in the gsan_factor coefficient and its loss of statistical significance when region variables are added suggests that the apparent effect of GSAN membership in the first model was likely due to differences between regions rather than the effect of GSAN membership itself.

_Regional Effects_: The region variables play a crucial role in explaining the likelihood of meeting the minimum diet diversity. Specifically, being in Douentza dramatically increases the odds, while being in Tombouctou decreases the odds significantly, compared to the reference region, Bandiagara.

### X. Logistic regression with  GSAN, region & Education as a predictor of mwdd

In [76]:
# Defining feature matrix X and target vector y
X2 = np.asarray(df_encoded[["gsan_factor_yes", "Region_Douentza", "Region_Gao", "Region_Tombouctou", "A15"]])
y2 = np.asarray(df_encoded['mwdd']) 

# Split the dataset into training and testing sets
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.3, random_state=42)

# Initialize SMOTE for resampling
smote = SMOTE(random_state=42)

# Resample the training data using SMOTE
X_train_res2, y_train_res2 = smote.fit_resample(X_train2, y_train2)

# Add a constant to the feature matrix for the intercept
X_train_with_const_res2 = sm.add_constant(X_train_res2)

# Convert the numpy array to a pandas DataFrame to use the replace method
X_train_with_const_res2_df = pd.DataFrame(X_train_with_const_res2)

# Replace infinite values with NaN
X_train_with_const_res2_df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Drop rows with NaNs from X_train_with_const_res2_df and corresponding y_train_res2 rows
X_train_with_const_res2_df.dropna(inplace=True)
y_train_res2 = y_train_res2[X_train_with_const_res2_df.index]

# Now run the logistic regression model
logit_model_res2 = sm.Logit(y_train_res2, X_train_with_const_res2_df)
result_res2 = logit_model_res2.fit()

# Print the summary of the logistic regression model
print(result_res2.summary())


Optimization terminated successfully.
         Current function value: 0.508654
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  626
Model:                          Logit   Df Residuals:                      620
Method:                           MLE   Df Model:                            5
Date:                Wed, 04 Sep 2024   Pseudo R-squ.:                  0.2662
Time:                        10:09:31   Log-Likelihood:                -318.42
converged:                       True   LL-Null:                       -433.91
Covariance Type:            nonrobust   LLR p-value:                 6.577e-48
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
0             -0.9644      0.238     -4.049      0.000      -1.431      -0.498
1              0.1415      0.

Testing the model

In [78]:
# Add constant to test data
X_test_with_const = sm.add_constant(X_test2)

# Predict probabilities on the test set
y_pred_prob = result_res2.predict(X_test_with_const)

# Convert predicted probabilities into binary predictions (threshold of 0.5)
y_pred = [1 if x >= 0.5 else 0 for x in y_pred_prob]

# Generate the classification report
print(classification_report(y_test2, y_pred))

              precision    recall  f1-score   support

           0       0.59      0.96      0.73        85
           1       0.95      0.53      0.68       120

    accuracy                           0.71       205
   macro avg       0.77      0.74      0.70       205
weighted avg       0.80      0.71      0.70       205



What the results show when gsan, region and education are used together as a predictor in meeting minimun diet ?

The consideration that we did in the previous model about the regional effect are still valid for this new model. We noticed that while the coefficient for GSAN membership has increased slightly it remains not statistically significant.

Higher levels of education are associated with an increased likelihood of meeting the minimum diet diversity. For each one-unit increase in education level (moving from one category to the next, e.g., from no education to basic literacy), the log-odds of meeting the minimum diet diversity increase by 0.1541. Education level is therefore a significant predictor, indicating that higher education levels are associated with better dietary outcomes, independent of region and GSAN membership.

### XI. Logistic regression with GSAN, Livelihood Zone & Education as a predictor of mwdd

In [81]:
# Defining feature matrix X and target vector y
X3 = np.asarray(df_encoded[["gsan_factor_yes", "LZ_cercle_ML02_ML03_ML04", "LZ_cercle_ML13", "LZ_cercle_ML13_ML09", "A15"]])
y3 = np.asarray(df_encoded['mwdd']) 

# Split the dataset into training and testing sets
X_train3, X_test3, y_train3, y_test3 = train_test_split(X3, y3, test_size=0.3, random_state=42)

# Initialize SMOTE for resampling
smote = SMOTE(random_state=42)

# Resample the training data using SMOTE
X_train_res3, y_train_res3 = smote.fit_resample(X_train3, y_train3)

# Add a constant to the feature matrix for the intercept
X_train_with_const_res3 = sm.add_constant(X_train_res3)

# Convert the numpy array to a pandas DataFrame to use the replace method
X_train_with_const_res3_df = pd.DataFrame(X_train_with_const_res3)

# Replace infinite values with NaN
X_train_with_const_res3_df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Drop rows with NaNs from X_train_with_const_res2_df and corresponding y_train_res2 rows
X_train_with_const_res3_df.dropna(inplace=True)
y_train_res3 = y_train_res2[X_train_with_const_res3_df.index]

# Now run the logistic regression model
logit_model_res3 = sm.Logit(y_train_res3, X_train_with_const_res3_df)
result_res3 = logit_model_res3.fit()

# Print the summary of the logistic regression model
print(result_res3.summary())


Optimization terminated successfully.
         Current function value: 0.513769
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  626
Model:                          Logit   Df Residuals:                      620
Method:                           MLE   Df Model:                            5
Date:                Wed, 04 Sep 2024   Pseudo R-squ.:                  0.2588
Time:                        10:09:31   Log-Likelihood:                -321.62
converged:                       True   LL-Null:                       -433.91
Covariance Type:            nonrobust   LLR p-value:                 1.550e-46
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
0             -3.6606      0.660     -5.548      0.000      -4.954      -2.367
1              0.0891      0.

In [82]:
# Add constant to test data
X_test_with_const = sm.add_constant(X_test3)

# Predict probabilities on the test set
y_pred_prob = result_res3.predict(X_test_with_const)

# Convert predicted probabilities into binary predictions (threshold of 0.5)
y_pred = [1 if x >= 0.5 else 0 for x in y_pred_prob]

# Generate the classification report
print(classification_report(y_test3, y_pred))

              precision    recall  f1-score   support

           0       0.59      0.96      0.73        85
           1       0.95      0.53      0.68       120

    accuracy                           0.71       205
   macro avg       0.77      0.74      0.70       205
weighted avg       0.80      0.71      0.70       205



__What the results show when gsan,livelihood zone and education are used together as a predictor in meeting minimun diet__ ?

_Effect of Adding livelihood zone Variables_: There is significant drop in the gsan_factor coefficient and a loss of statistical significance when livelihood variable are added. This drop is even greater in comparsion to when we added the region.  Again, this  suggests that the apparent effect of GSAN membership in the first model was likely due to differences between livelihood zone  rather than the effect of GSAN membership itself.

_Livelihoodal Effec_ts: ThLivelihood zoneon variables play a crucial role in explaining the likelihood of meeting the minimum diet diversity. Specificy,y, being ibeign in ML13_ML09 (Koro), y increases the od more , compared to the referenc livelihood zone ML02_ML03 in Tombouctoura.

The stark differences in the likelihood of meeting minimum diet diversity acrthe livelihoodsions indicate that local contxors, may play a crucial role in food groups availability and influencing women dietary outcomes. This suggest that the effectiveness of GSAN (Groupement de Santé et d'Action Nutritionnelle) may vary significantlylivelihoodgion. This means that a one-size-fits-all strategy for implementing GSAN might not be equally effective across differlivelihoodions. The analysis indicates that a region-livelihoodally adaptable GSAN strategy is likely necessary to maximize its impact. Tailoring the approach to the unique circumstances of elivelihood zonegion can help ensure that GSAN initiatives are more effective and equitable, ultimately improving dietary outcomes for women across different ar

For education the observation in the previous modele is still valide. eas.

### XI. What are the next steps ?

Taken alone, with no other variable in the model, we have sufficient evidence for responding the questions in the project. With the multivariate regression,  we noticed an accuracy of 0.70, meaning the model is correctly classifying only 70% of the instances. In fields of social science or marketing  where the consequences of a wrong prediction are less severe, an accuracy of around 0.70 might be considered acceptable, especially if the model provides actionable insights or is being used as part of a broader decision-making process. Including additional variables such as access to cash via UCT and income could be beneficial for your model, providing deeper insights into the factors influencing dietary diversity.  These variable will be available for the FY24 survey and these economic factors are likely to play a significant role in determining whether women can meet minimum diet diversity standards, and their inclusion could enhance the explanatory power and practical relevance of your analysis. Alternatively, model Tuning by experimenting  with different algorithms (such as decision tree) or hyperparameters, cross validation  to improve the model's performance can be also next steps.